# Batch processing
Tobias Rose 2020

# Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import seaborn; seaborn.set()  # set plot styles
import sys
import os
from pathlib import Path
from scipy.signal import convolve
#from skimage.viewer import ImageViewer
from tqdm import tqdm

### User-specific folders

In [ ]:
if sys.platform == "darwin":
    ### Mac
    main_root = '/Volumes/archive_bonhoeffer_group$/David Laubender/Data/imaging data/DL_191024_6/ImagingData/' #location of original data
    adata     = '/Volumes/archive_bonhoeffer_group$/David Laubender/adata' #location of saved analyzed data
    ftemp     = '/Users/trose/Data/temp' #fast disk (local ssd for s2p binary files) 
    ftiff     = '/Users/trose/Data/s2p_tiff' #fast disk folder for concatenated tiffs (if needed)
elif sys.platform == "win32":
    main_root = 'I:/David Laubender/Data/imaging data/DL_191024_6/ImagingData' #location of original data
    adata     = 'I:/David Laubender/adata' #location of saved analyzed data
    ftemp     = 'C:/temp/trose/suite2ptemp' #fast disk (local ssd for s2p binary files  
    ftiff     = 'C:/temp/trose/s2p_tiff' #fast disk folder for concatenated tiffs (if needed)

In [ ]:
exp = ['62283', '62284', '62285', '62286', '62287', '62288', '62289', '62290']

In [ ]:
exp = ['62284']

In [ ]:
runops = {
    'exp': exp, 
    'concat': False,               #whether or not to run all experiments as a concatenated single experiment
    'main_root': main_root,
    'adata': adata,
    'darkframes': 50,              #number of darkframes to be excluded from extraction
    'readfiles':10
}

In [ ]:
adata_s2ppath = []
adata_roipath = []
aux_files = []

for val in exp:
    files = list(Path(main_root).rglob('suite2p_exp'+val+'/')) #recursive
    print(files)
    try:
        adata_s2ppath.append(os.path.join(files[0], 'suite2p', 'combined'))
        adata_roipath.append(os.path.dirname(files[0]))
        aux_files.append(*Path(os.path.join(*Path(files[0]).parts[0:-3], 'data', Path(files[0]).parts[-2])).glob('exp'+val+'*.lvd'))
        #aux_files.append = list(Path(aux_data_path).glob('exp'+val+'*')) #recursive
    except:
        print(val + ' not found')
        
#adata_s2ppath = sorted(adata_s2ppath)

In [ ]:
F = np.load(os.path.join(adata_s2ppath[0],'F.npy'))
Fneu = np.load(os.path.join(adata_s2ppath[0],'Fneu.npy'))
spks = np.load(os.path.join(adata_s2ppath[0],'spks.npy'))
stat = np.load(os.path.join(adata_s2ppath[0],'stat.npy'), allow_pickle=True)
ops = np.load(os.path.join(adata_s2ppath[0],'ops.npy'), allow_pickle=True).item()
iscell = np.load(os.path.join(adata_s2ppath[0],'iscell.npy'))

### Preliminary aux loader

In [ ]:
filename = str(aux_files[0])

In [ ]:
def load_auxdata(filename):
    """ Loads .lvd aux data file """
    with open(filename, 'rb') as f:
        # Reset file index
        f.seek(0)
        # Get meta data
        samplingfreq = np.fromfile(f, dtype='>f8', count=1)
        print("Aux sampling frequency = {}Hz".format(samplingfreq))
        n_channels = int(np.fromfile(f, dtype='>f8', count=1))
        print("# channels = {}".format(n_channels))
        timestamp = np.fromfile(f, dtype='>f8', count=1)
        print("timestamp = {}".format(timestamp))
        max_input = np.fromfile(f, dtype='>f8', count=1)
        print("max input = {} V".format(max_input))
        # Read aux data
        auxdata = np.fromfile(f, dtype='>f8')
        n_datapoints = int(auxdata.shape[0]/n_channels)
        print("number of aux datapoints = {}".format(n_datapoints))
        auxdata = np.reshape(auxdata,(n_datapoints,n_channels))
        return auxdata, samplingfreq

In [ ]:
def get_frame_times(auxdata, Frames_chan):
    """ extracts frame onset times """
    
    pos = np.argwhere(auxdata[0:,Frames_chan] > 0.75 * np.max(auxdata[range(0,len_aux),Frames_chan ])) # work on diff of indices rather than on raw diff to prevent multi-smaple detection in up/ downstrokes
    diffpos = np.argwhere(np.diff(pos[0:,0]) > 1)
    frame_times = pos[diffpos,0]
    
    if  len(frame_times)==0:
        print('get_frame_times WARNING: no frames found')
        frame_times = 1;
        return frame_times
    
    # find onset of first frame
    pos_first = np.argwhere(auxdata[0:,Frames_chan] < 0.5 * np.max(auxdata[range(0,len_aux), Frames_chan]))
    diffpos_first = np.argwhere(np.diff(pos_first[0:,0]) > 1)
    frame_times = np.append(diffpos_first[0], frame_times)
    
    return frame_times

In [ ]:
[auxdata, samplingfreq] = load_auxdata(filename)

In [ ]:
Frames_chan = 3
Stims_chan = 7
eye1_chan = 16
eye2_chan = 17

len_aux = len(auxdata)
step = 5

level = 4 # extract from SI file in the future

In [ ]:
# frame_times = get_frame_times(auxdata, Frames_chan)

### plot aux_data

In [ ]:
fig = plt.figure(figsize=(16,8))
plt.subplot(4,1,1)
plt.plot(auxdata[range(0,len_aux,step),Frames_chan])
plt.subplot(4,1,2)
plt.plot(auxdata[range(0,len_aux,step),Stims_chan])
plt.subplot(4,1,3)
plt.plot(auxdata[range(0,len_aux,step),eye1_chan])
plt.subplot(4,1,4)
plt.plot(auxdata[range(0,len_aux,step),eye2_chan])
plt.show

In [ ]:
frame_times = get_frame_times(auxdata, Frames_chan)

In [ ]:
fig = plt.figure(figsize=(8,4))
plt.plot(auxdata[range(0,30000,1),Frames_chan])
plt.vlines(frame_times[0:20],-1,6)

### extract chirp stim timebase

In [350]:
frame_times = get_frame_times(auxdata, Frames_chan)
frame_times_level   = frame_times[range(0,len(frame_times),level)]
minsample_delta = 100 #minimum stim duration in level frames

In [392]:
stimops = {
    'Frames_chan': 3,
    'Stims_chan': 7,
    'eye1_chan': 16,
    'eye2_chan': 17,
    'len_aux': len(auxdata),
    'step': 5,
    'level': 4, # extract from SI file in the future
    'minsample_delta': 100 
     }

In [ ]:
fig = plt.figure(figsize=(16,12))
plt.subplot(6,1,1)
plt.plot(Eye2On), plt.ylabel('Eye2On') 
plt.subplot(6,1,2)
plt.plot(Eye1On), plt.ylabel('Eye1On') 
plt.subplot(6,1,3)
plt.plot(bino), plt.ylabel('bino') 
plt.subplot(6,1,4)
plt.plot(Eye1On_only), plt.ylabel('Eye1On_only') 
plt.subplot(6,1,5)
plt.plot(Eye2On_only), plt.ylabel('Eye2On_only') 
plt.subplot(6,1,6)
plt.plot(StimOn), plt.ylabel('StimOn') 

In [393]:
def get_stimIDs_chirp(auxdata, stimops):
    """ extracts chirp onset times """
    
    Frames_chan = stimops['Frames_chan']
    Stims_chan = stimops['Stims_chan']
    eye1_chan = stimops['eye1_chan']
    eye2_chan = stimops['eye2_chan']
    level = stimops['level']
    minsample_delta = stimops['minsample_delta']
    
    frame_times         = get_frame_times(auxdata,Frames_chan)
    frame_times_level   = frame_times[range(0,len(frame_times),level)]

    
    StimOn = auxdata[frame_times_level, Stims_chan]>0.8

    # generate cleaned eye binaries
    Eye1On = auxdata[frame_times_level,eye1_chan]*-1+ np.max(auxdata[frame_times_level,eye1_chan])>0.8
    Eye2On = auxdata[frame_times_level,eye2_chan]>0.8
    Eye2On[-1] = 1 
    bino = Eye1On == Eye2On

    Eye1On_only = Eye1On != bino
    Eye2On_only = Eye2On != bino
    Eye1On_only[-1] = False
    Eye2On_only[-1] = False
    bino[0] = False
    bino[-1] = False

    # generate cleaned bino binary
    bino_onsets_temp  = np.argwhere(np.diff(np.multiply(bino, 1)) > 0)
    bino_offsets_temp = np.argwhere(np.diff(np.multiply(bino, 1)) < 0)

    bino_onsets  = bino_onsets_temp[np.argwhere(bino_offsets_temp[0:,0] - bino_onsets_temp[0:,0] > minsample_delta)]
    bino_offsets = bino_offsets_temp[np.argwhere(bino_offsets_temp[0:,0] - bino_onsets_temp[0:,0] > minsample_delta)]                              

    bino_clean = np.full(( len(frame_times_level)), False) 

    for i in range(len(bino_onsets)):
        bino_clean[range(bino_onsets[i,0,0], bino_offsets[i,0,0])] = True

    # extract chirp stim on and offsets
    chirp_onsets_temp  = np.argwhere(np.diff(np.multiply(StimOn, 1)) > 0)
    chirp_offsets_temp = np.argwhere(np.diff(np.multiply(StimOn, 1)) < 0)

    chirp_on  = np.argwhere(np.diff(chirp_onsets_temp[0:,0]) > minsample_delta) + 1
    chirp_off = np.argwhere(np.diff(chirp_offsets_temp[0:,0]) > minsample_delta)

    chirp_on  = np.append(0, chirp_on)
    chirp_off = np.append(chirp_off, len(chirp_offsets_temp) - 1)

    chirp_onsets  = chirp_onsets_temp[chirp_on]
    chirp_offsets = chirp_offsets_temp[chirp_off]

    ids = { 
    'StimBoundsIpsi':   [np.intersect1d(chirp_onsets, np.argwhere(Eye1On_only)), np.intersect1d(chirp_offsets, np.argwhere(Eye1On_only))],
    'StimBoundsContra': [np.intersect1d(chirp_onsets, np.argwhere(Eye2On_only)), np.intersect1d(chirp_offsets, np.argwhere(Eye2On_only))],
    'StimBoundsBino':   [np.intersect1d(chirp_onsets, np.argwhere(bino_clean)), np.intersect1d(chirp_offsets, np.argwhere(bino_clean))]
    }
    
    return ids
    

In [394]:
get_stimIDs_chirp(auxdata, stimops)

{'StimBoundsIpsi': [array([ 877, 3072, 3438, 4169, 4901, 5998, 6364, 8192], dtype=int64),
  array([1114, 3308, 3674, 4405, 5137, 6234, 6600, 8428], dtype=int64)],
 'StimBoundsContra': [array([ 146, 1243, 1609, 1975, 3803, 4535, 5266, 7461], dtype=int64),
  array([ 382, 1479, 1845, 2211, 4040, 4771, 5503, 7697], dtype=int64)],
 'StimBoundsBino': [array([ 512, 2340, 2706, 5632, 6729, 7095, 7827, 8558], dtype=int64),
  array([ 748, 2577, 2942, 5868, 6965, 7331, 8063, 8794], dtype=int64)]}

In [367]:
fig = plt.figure(figsize=(8,6))
plt.plot(StimOn)
plt.vlines(chirp_onsets[0:],-1,2,'r')
plt.vlines(chirp_offsets[0:2],-1,2,'k')

C:\Miniconda3\envs\suite2p\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [368]:
fig = plt.figure(figsize=(8,6))
plt.plot(bino)
plt.vlines(bino_offsets[0:],-1,2,'k')
plt.vlines(bino_onsets[0:],-1,2,'r')

C:\Miniconda3\envs\suite2p\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### plot examples

In [ ]:
# show cells
im = np.zeros((ops['Ly'], ops['Lx']))
ncells = len(stat)

for n in range(0,ncells):
    ypix = stat[n]['ypix'][~stat[n]['overlap']]
    xpix = stat[n]['xpix'][~stat[n]['overlap']]
    im[ypix,xpix] = n+1
fig = plt.figure(figsize=(7,4))
plt.imshow(im)
plt.tight_layout()
plt.show()

In [ ]:
ops['tau'] = .7

In [ ]:
# show mean image
fig = plt.figure(figsize=(8,3))
plt.subplot(1,3,1)
plt.imshow(ops["meanImg"])
plt.subplot(1,3,2)
plt.imshow(ops["meanImgE"])
plt.subplot(1,3,3)
plt.imshow(ops["Vcorr"])
plt.show

In [ ]:
ops['tau'] = 0.5
bouton = 11;

In [ ]:
efilt = np.exp(- np.linspace(0,50,200) / (ops['tau'] * ops['fs']))
#efilt /= efilt.sum()
sout = convolve(spks[bouton,:], efilt)
sout = sout[:spks.shape[1]]

In [ ]:
fig = plt.figure(figsize=(16,4))
plt.plot(F[bouton]-Fneu[bouton] * .7)
plt.show

In [ ]:
plt.plot(sout)
plt.tight_layout()
plt.show
#plt.plot(F[10:])

In [ ]:
plt.plot(spks[11])
plt.show

In [ ]:
plt.figure(figsize=(7,4))
plt.imshow(spks[:100, :5000], vmax = 3, vmin = -0.5, aspect='auto', cmap = 'gray_r')
plt.title('sample of the neural data matrix')
plt.ylabel('boutons') 
plt.xlabel('time [samples]')

In [ ]:
ops

# ToDo

- batch run different deconvolution settings using this code snippet

In [ ]:
# compute deconvolution
from suite2p import dcnv
import numpy as np

tau = 1.0 # timescale of indicator
fs = 30.0 # sampling rate in Hz
neucoeff = 0.7 # neuropil coefficient
# for computing and subtracting baseline
baseline = 'maximin' # take the running max of the running min after smoothing with gaussian
sig_baseline = 10.0 # in bins, standard deviation of gaussian with which to smooth
win_baseline = 60.0 # in seconds, window in which to compute max/min filters

ops = {'tau': tau, 'fs': fs, 'neucoeff': neucoeff,
       'baseline': baseline, 'sig_baseline': sig_baseline, 'win_baseline': win_baseline}

# load traces and subtract neuropil
F = np.load('F.npy')
Fneu = np.load('Fneu.npy')
Fc = F - ops['neucoeff'] * Fneu

# baseline operation
Fc = dcnv.preprocess(Fc, ops)

# get spikes
spks = dcnv.oasis(Fc, ops)